# Bike Sharing Demand：自転車利用数予測
https://www.kaggle.com/c/bike-sharing-demand
## Washington, D.C. の過去のレンタル自転車の利用パターン情報と気象データから、レンタル自転車の利用数を予測するモデルを作成する

### 評価指標：RMSLE
$$
\sqrt{\frac{1}{n}\sum_{i=0}^{n}(\log(p_i+1)-\log(a_i+1))^2}
$$
### data field
**datetime** - hourly date + timestamp  
**season** -  1 = spring, 2 = summer, 3 = fall, 4 = winter  
**holiday** - whether the day is considered a holiday  
**workingday** - whether the day is neither a weekend nor holiday  
**weather** - 1: Clear, Few clouds, Partly cloudy, Partly cloudy  
2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist  
3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds  
4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog  
**temp** - temperature in Celsius  
**atemp** - "feels like" temperature in Celsius  
**humidity** - relative humidity  
**windspeed** - wind speed  
**casual** - number of non-registered user rentals initiated  
**registered** - number of registered user rentals initiated  
**count** - number of total rentals  

In [ ]:
#%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, ParameterGrid
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

### データの読み込み関連
ここら辺は特段難しいことはやってないので、基本コピペで問題ないです。  
C言語触ったことある人だったら、初見でもなんとなくやってることはわかるのではと思います。

In [ ]:
TRAIN_DATA = '../input/train.csv'
TEST_DATA = '../input/test.csv'

parse_datesで指定した列をdatetime型で読み込んでおくと、後々便利

In [ ]:
def read_csv(path):
    df = pd.read_csv(path, parse_dates=[0])
    return df

In [ ]:
def load_train_data():
    df = read_csv(TRAIN_DATA)
    return df

In [ ]:
def load_test_data():
    df = read_csv(TEST_DATA)
    return df

In [ ]:
df = load_train_data()
test = load_test_data()

### データの確認

In [ ]:
df.head()

In [ ]:
test.head()

In [ ]:
print(df.shape)
print(test.shape)

In [ ]:
df.info()

今回のデータは欠損値がありません。素晴らしい。

In [ ]:
df.isnull().sum()

## feature engineering
datetimeを年月日、時間に分割

In [ ]:
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['day'] = df['datetime'].dt.day
df['dayofweek'] = df['datetime'].dt.dayofweek
df['hour'] = df['datetime'].dt.hour

In [ ]:
test['year'] = test['datetime'].dt.year
test['month'] = test['datetime'].dt.month
test['day'] = test['datetime'].dt.day
test['dayofweek'] = test['datetime'].dt.dayofweek
test['hour'] = test['datetime'].dt.hour

### 評価の際に、scikit-learnに用意されているrmse関数を使用するため、logに変換しておく

In [ ]:
df['count'] = np.log(df['count'] + 1)
df.rename(columns={'count':'rentals'}, inplace=True)

In [ ]:
df.head()

### モデル作成
まずは何も考えず、ランダムフォレストでやってみる

In [ ]:
removed_cols = ['rentals', 'casual', 'registered', 'datetime']
feats = [c for c in df.columns if c not in removed_cols]
x_train = df[feats]
y_train = df['rentals'].values

In [ ]:
clf = RandomForestRegressor(random_state=0)
clf.fit(x_train, y_train)

### 予測精度の評価

In [ ]:
def logmse(y, pred):
    g = mean_squared_error(y, pred)**(1/2)
    return g

In [ ]:
y_pred = clf.predict(x_train)
print(logmse(y_train, y_pred))

あとはテストデータに対してもpredictして提出用ファイルを作成すればOKです。
提出用ファイルの作り方は割愛。  
githubに色々とコード載せているので、興味ある方は参照ください。  
https://github.com/msk0119/kaggle-bike/blob/master/xgb_cv5.py

### 評価方法
上で算出した予測精度は、学習データに対する評価スコア。  
実際には未知データに対して予測する必要があるため、学習データでテストしてもモデルの汎化能力は評価できない。  
そのため、学習データのうち、いくつかのデータを評価用データとする手法で評価を行うのが一般的。
#### ホールドアウト法

In [ ]:
x_trn, x_val, y_trn, y_val = train_test_split(x_train, y_train, test_size=0.30, shuffle = False)#時系列データのためshuffleしない方がいいです。
clf = RandomForestRegressor(random_state=0)
clf.fit(x_trn, y_trn)
y_trn_pred = clf.predict(x_trn)
y_val_pred = clf.predict(x_val)
print("trainスコア", logmse(y_trn, y_trn_pred))
print("validスコア", logmse(y_val, y_val_pred))

#### クロスバリデーション

In [ ]:
kf = KFold(n_splits=3, shuffle=False)

list_logmse_score = []

for train_idx, valid_idx in kf.split(x_train, y_train):
    trn_x = x_train.iloc[train_idx, :]
    val_x = x_train.iloc[valid_idx, :]
    
    trn_y = y_train[train_idx]
    val_y = y_train[valid_idx]
    
    clf = RandomForestRegressor(random_state=0)
    clf.fit(trn_x, trn_y)
    y_pred = clf.predict(val_x)
    
    sc_logmse = logmse(val_y, y_pred)
    
    list_logmse_score.append(sc_logmse)
    
sc_logmse = np.mean(list_logmse_score)
print(list_logmse_score)
print(sc_logmse)

以上で正しい評価スコアが算出できるため、評価スコアが向上するように、序盤はひたすらにfeature engineeringを頑張るべき。  
ある程度いい評価スコアを得られる変数を特定できれば、終盤はハイパーパラメータの探索、モデルのアンサンブル等を行う。

### ハイパーパラメータの探索

In [ ]:
kf = KFold(n_splits=3, shuffle=False, random_state=0)

all_params = {'max_depth': [3, 5, 7],
              "n_estimators": [10, 100],
              'min_samples_split':[2, 4],
              'min_samples_leaf':[1, 3],
            'random_state':[0]}

min_score = 100
min_params = None

for params in tqdm(list(ParameterGrid(all_params))):
    
    list_logmse_score = []

    for train_idx, valid_idx in kf.split(x_train, y_train):
        trn_x = x_train.iloc[train_idx, :]
        val_x = x_train.iloc[valid_idx, :]
    
        trn_y = y_train[train_idx]
        val_y = y_train[valid_idx]
    
        clf = RandomForestRegressor(**params)
        clf.fit(trn_x, trn_y)
        y_pred = clf.predict(val_x)
    
        sc_logmse = logmse(val_y, y_pred)
    
        list_logmse_score.append(sc_logmse)

    sc_logmse = np.mean(list_logmse_score)
    if min_score > sc_logmse:
        min_score = sc_logmse
        min_params = params
        
print('minimam logmse:',min_score)